In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from itertools import count
import csv
import sys
import re
import time
from tqdm import tqdm_notebook
sys.path.insert(0,'C:\\Users\\axelv\\Downloads\\chromedriver_win32')

In [2]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--start-maximized")
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

browser = webdriver.Chrome(executable_path='C:\\Users\\axelv\\Downloads\\chromedriver_win32\\chromedriver.exe',options=chrome_options)
browser.delete_all_cookies()
browser.set_window_position(0,0)

{'height': 980, 'width': 1050, 'x': 0, 'y': 0}

In [3]:
URL_SUFFIX = "te-koop/gent/9000"

In [4]:
def iterate_pages():
    for i in tqdm_notebook(count(1), desc="Page"):
        div_list = list()
        try_counter = 0
        while not div_list or try_counter > 10:
            browser.get("https://www.immoweb.be/nl/zoek/appartement/"+URL_SUFFIX+"?page=%d"%i)
            html = BeautifulSoup(browser.page_source, 'html.parser')
            div_list = html.findAll("div", {"data-type": re.compile("resultgallery")})
            try_counter += 1
        if div_list:
            for div_item in div_list:
                
                yield div_item
                time.sleep(2)

In [5]:
def load_address(id_str:str):
    while True:
        try:
            browser.get("https://www.immoweb.be/nl/zoekertje/appartement/"+URL_SUFFIX+"/id%s"%id_str)
            html = BeautifulSoup(browser.page_source, 'html.parser')
            address = html.find("a", {"id":re.compile("address")}).text.replace(",","")
            return address
        except AttributeError:
            time.sleep(5)    

In [6]:
def parse_record(div):
    record = dict()
    record["price"] = div.find("div", {"class": re.compile("rangePrice")}).text.strip()[:6]
    l = div.find("div", {"class": re.compile("surface")}).text.strip().split("\xa0")
    try:
        record["opp"] = l[0]
        record["rooms"] = l[3]
    except IndexError:
        record["opp_rooms"] = l[0]
    record["city"] = div.find("span", {"class": "result-adress"}).text
    
    record["address"] = load_address(div["id"])
    record = dict(map(lambda s: (s[0], re.sub('\s+',' ',s[1]).strip()), record.items()))
    return record

In [7]:
div_iter = iterate_pages()

In [8]:
try:
    with open("data.csv", 'w', newline='\n') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["price", "opp", "rooms", "opp_rooms", "city", "address"])
        writer.writeheader()
        for i,div in tqdm_notebook(enumerate(div_iter)):
            record_dict = parse_record(div)
            writer.writerow(record_dict)
            if i % 100 == 0:
                csvfile.flush()

except IOError:
    print("I/O error")

KeyboardInterrupt: 

In [9]:
csvfile.flush()

ValueError: I/O operation on closed file.